In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

import autograd
import autograd.numpy as np

import scipy.integrate
solve_ivp = scipy.integrate.solve_ivp

import csv

import requests

from functions import *
from tqdm import tqdm


# API call

In [2]:
# https://ssd-api.jpl.nasa.gov/doc/horizons.html

url = "https://ssd.jpl.nasa.gov/api/horizons.api"
format = "format=text"
command = "COMMAND=" # target body
targets = ["10", "199", "299", "399", "499", "599", "699", "799", "899"]  # sun, mercury, venus, earth, mars, jupiter, saturn, uranus, neptun
target_names = ["sun", "mercury", "venus", "earth", "mars", "jupiter", "saturn", "uranus", "neptun"]
obj_data = "OBJ_DATA='NO'"
ephemeris = "MAKE_EPHEM='YES'"
eph_type = "EPHEM_TYPE='VECTORS'"
vec_table = "VEC_TABLE='2'"
center = "CENTER='500@0'"
start = "START_TIME='2019-01-01'"
stop = "STOP_TIME='2023-01-01'"
step = "STEP_SIZE='1 h'"
cal_type = "CAL_TYPE=GREGORIAN"
vector_labels = "VEC_LABELS=NO"
csv_format = "CSV_FORMAT=YES"
units = "OUT_UNITS='AU-D'"
#quantities = "QUANTITIES='1,9,20,23,24,29'"

In [3]:
targets = targets[1:3]
target_names = target_names[1:3]

In [4]:
data = []
time = []
for target in targets:
    path = url+"?"+format+"&"+command+target+"&"+obj_data+"&"+ephemeris+"&"+eph_type+"&"+vec_table+"&"+center+"&"+start+"&"+stop+"&"+step+"&"+cal_type+"&"+vector_labels+"&"+csv_format+"&"+units
    q, p, t = read_data(requests.get(path).text)
    data.append([[q], [p]])
    time.append([t])
    
data = np.asarray(data).reshape((len(targets),2,len(data[0][0][0]),3))
time = np.asarray(time).reshape((len(targets),len(time[0][0])))

In [ ]:
print(data.shape)

## Data shape

9 x target body <br>
2 x vectors - q & p <br>
n x data points <br>
3 x coordinates - x, y, z

# Phase spaces for all bodies in the system

In [ ]:
plot_phase_space(data, target_names, "Phase space of the solar system")

In [ ]:
plot_space(data, target_names, "Space of the solar system")

# Learning the solar system

## Preparing the data

In [ ]:
q, dq, p, dp, max_q, max_p = prep_data(data, time)

q_p = np.concatenate((q, p), axis=1)
q_p = torch.tensor(q_p, requires_grad=True, dtype=torch.float32)

dq_dp = np.concatenate((dq, dp), axis=1)
dq_dp = torch.tensor(dq_dp, requires_grad=True, dtype=torch.float32)

In [ ]:
#torch.save(q_p, "data/q_p.pt")
#torch.save(dq_dp, "data/dq_dp.pt")
#np.save("data/time.npy", time)
#np.save("data/data.npy", data)

## Normal Neural Network

In [ ]:
class MLP(nn.Module):
    def __init__(self, bodys) -> None:
        super().__init__()

        self.fc1 = nn.Linear(bodys, 500)
        self.fc2 = nn.Linear(500, 200)
        self.fc3 = nn.Linear(200, 200)
        self.fc4 = nn.Linear(200, bodys)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

## Hamiltonian

## Training loop

In [ ]:
def train(model, x, dx, batch_size=32, epochs=100, baseline=True, lr=0.001):
    loss_hist = []
    ep_hist = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss = nn.MSELoss()

    for step in tqdm(range(epochs)):
        model.train()

        batches = np.random.choice(len(x), len(x))

        for batch in range(0, len(x)//batch_size+1):

            low = batch * batch_size
            high = min((batch + 1) * batch_size, len(x))
            bb = batches[low:high]

            x_batch = x[bb]
            dx_batch = dx[bb]

            optimizer.zero_grad()

            if baseline:
                pred = model(x_batch)
            else:   
                pred = HNN(x_batch, model)

            loss_val = loss(pred, dx_batch)
            loss_val.backward()
            optimizer.step()

        if step % (epochs/10) == 0:
            model.eval()
            if baseline:
                pred = model(x)
            else:   
                pred = HNN(x, model)

            loss_val = loss(pred, dx)
            loss_hist.append(loss_val.item())
            ep_hist.append(step+1)
            
            #print(f"Epoch {step}/{epochs} --- Train-Loss: {loss_val.item()}")#{round(loss_val.item(), 3)}")

    plt.plot(ep_hist, loss_hist)

# Baseline NN

In [ ]:
model_baseline = MLP(q_p.shape[1])

train(model=model_baseline, x=q_p, dx=dq_dp, batch_size=32, epochs=30, baseline=True, lr=1e-3)


In [ ]:

#torch.save(model_baseline.state_dict(), "data/model_baseline.pt")

# HNN

In [ ]:
model_HNN = MLP(q_p.shape[1])

train(model=model_HNN, x=q_p, dx=dq_dp, batch_size=32, epochs=30, baseline=False, lr=1e-3)

#torch.save(model_HNN.state_dict(), "data/model_HNN.pt")

# Predicting

In [ ]:
q_p0 = q_p[0].detach().numpy()
t_span = [0, 18000]
steps = 100000

# integrate
t = torch.linspace(t_span[0], t_span[1], steps)
xHNN = integrate_model(model_HNN.double(), t_span=t_span, y0=q_p0, t_eval=t, baseline=False)
xBaseline = integrate_model(model_baseline.double(), t_span=t_span, y0=q_p0, t_eval=t, baseline=True)

In [ ]:
# rescale output to original values

xHNN.y[0:(len(targets)*3)] = xHNN.y[0:(len(targets)*3)]*max_q
xHNN.y[(len(targets)*3):len(xHNN.y)] = xHNN.y[(len(targets)*3):len(xHNN.y)]*max_p

xBaseline.y[0:(len(targets)*3)] = xBaseline.y[0:(len(targets)*3)]*max_q
xBaseline.y[(len(targets)*3):len(xBaseline.y)] = xBaseline.y[(len(targets)*3):len(xBaseline.y)]*max_p

In [ ]:
HNN_data = reshape_data(xHNN.y)
Baseline_data = reshape_data(xBaseline.y)

# Plot the results
## HNN

In [ ]:
plot_phase_space(HNN_data, target_names, "Phase spaces for HNN")

In [ ]:
plot_space(HNN_data, target_names, "HNN", color=False)

## Baseline NN

In [ ]:
plot_phase_space(Baseline_data, target_names, "Phase spaces for Baseline")

In [ ]:
plot_space(Baseline_data, target_names, "Baseline")